# Conversational Threads

Many LLM applications have a chatbot-like interface in which the user and the LLM application engage in a multi-turn conversation. In order to track these conversations, you can use the Threads feature in LangSmith.

This is relevant to our RAG application, which should maintain context from prior conversations with users.

### Setup

In [1]:
# Load environment from ../../.env
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

False

In [2]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

False

### Group traces into threads


A Thread is a sequence of traces representing a single conversation. Each response is represented as its own trace, but these traces are linked together by being part of the same thread.

To associate traces together, you need to pass in a special metadata key where the value is the unique identifier for that thread.

The key value is the unique identifier for that conversation. The key name should be one of:

- session_id
- thread_id
- conversation_id.

The value should be a UUID.

In [3]:
import uuid
thread_id = uuid.uuid4()

In [4]:
from langsmith import traceable
from typing import List
import nest_asyncio
import os, sys
sys.path.append(os.path.abspath(".."))
import google.generativeai as genai
from module_0.utils import get_vector_db_retriever

genai.configure(api_key=os.getenv("GEMINI_API_KEY", ""))
nest_asyncio.apply()
retriever = get_vector_db_retriever()

@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    rag_system_prompt = """You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the latest question in the conversation. 
    If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise.
    """
    messages = [
        {
            "role": "system",
            "content": rag_system_prompt
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

@traceable(run_type="llm")
def call_openai(
    messages: List[dict], model: str = "models/gemini-2.5-flash", temperature: float = 0.0
) -> str:
    prompt = "\n\n".join([m["content"] for m in messages])
    model = genai.GenerativeModel(model)
    resp = model.generate_content(prompt, generation_config={"temperature": temperature})
    class Msg:
        def __init__(self, content): self.content = content
    class Choice:
        def __init__(self, content): self.message = Msg(content)
    class Resp:
        def __init__(self, content): self.choices = [Choice(content)]
    return Resp(getattr(resp, "text", ""))

@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


USER_AGENT environment variable not set, consider setting it to identify your requests.


### Now let's run our application twice with this thread_id

In [5]:
question = "Nischala: How do I add metadata to a Trace?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"thread_id": thread_id}})
print(ai_answer)

You can add metadata to traces by providing a dictionary of key-value pairs. This is done through the `experimental_telemetry` object, where you set `isEnabled` to true and include your metadata. For example, you can add `userId` and `language` as metadata when calling `generateText`.


In [6]:
question = "For Nischala: How can I add tags to a Trace?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"thread_id": thread_id}})
print(ai_answer)

You can add tags to a trace by providing them in the `RunnableConfig`. These tags are strings used to categorize or label a trace. When attached to a runnable, they are inherited by all its child runnables.


### Let's take a look in LangSmith!